In [1]:
import caffe

In [2]:
import facenet
import os
import tensorflow as tf

In [3]:
def copy_conv_layer_params(tf_names, caffe_names, caffe_net, model_dir = '/home/ovuser/FaceRecognition/models/20170512-110547'):
    with tf.Graph().as_default():
        with tf.Session() as sess:
            facenet.load_model(model_dir)
            all_tensors = tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
            all_tensor_names = [tensor.name for tensor in all_tensors]
            for i in range(len(caffe_names)):
                caffe_name = caffe_names[i]
                tf_name = tf_names[i]
                tf_name_index = all_tensor_names.index(tf_name)
                tensor = all_tensors[tf_name_index]
                print ('Copying ' + tensor.name+' into '+caffe_name)
                value = tensor.eval()
                print ('tf_tensor.shape: ', value.shape)
                caffe_net.params[caffe_name][0].data[...] = value.transpose((3,2,1,0))
            return caffe_net

In [9]:
caffe_net = copy_conv_layer_params(list(conv_layers.keys()), list(conv_layers.values()), caffe_net)

Model directory: /home/ovuser/FaceRecognition/models/20170512-110547
Metagraph file: model-20170512-110547.meta
Checkpoint file: model-20170512-110547.ckpt-250000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /home/ovuser/FaceRecognition/models/20170512-110547/model-20170512-110547.ckpt-250000


/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Copying InceptionResnetV1/Repeat/block35_4/Branch_2/Conv2d_0b_3x3/weights:0 into inception_a4_3x3_b2
('tf_tensor.shape: ', (3, 3, 32, 32))
Copying InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0b_3x3/weights:0 into inception_a3_3x3_b1
('tf_tensor.shape: ', (3, 3, 32, 32))
Copying InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/weights:0 into inception_a1_3x3_b2
('tf_tensor.shape: ', (3, 3, 32, 32))
Copying InceptionResnetV1/Repeat/block35_2/Branch_1/Conv2d_0b_3x3/weights:0 into inception_a2_3x3_b1
('tf_tensor.shape: ', (3, 3, 32, 32))
Copying InceptionResnetV1/Repeat/block35_5/Branch_2/Conv2d_0a_1x1/weights:0 into inception_a5_1x1_b2
('tf_tensor.shape: ', (1, 1, 256, 32))
Copying InceptionResnetV1/Repeat/block35_3/Conv2d_1x1/weights:0 into inception_a3_1x1
('tf_tensor.shape: ', (1, 1, 96, 256))
Copying InceptionResnetV1/Mixed_6a/Branch_1/Conv2d_1a_3x3/weights:0 into reduction_a_conv_3x3_3
('tf_tensor.shape: ', (3, 3, 192, 256))
Copying InceptionResnetV1/Repeat/block35_4/

In [5]:
caffe_tensors = []
nodes = caffe_net.params.keys()
for node in nodes:
    for node_layer in caffe_net.params[node]:
        shape = node_layer.data.shape
        shape_str = ','.join([str(it) for it in shape])
        caffe_tensors.append(node+'\n')

In [6]:
with open('/home/ovuser/Projects/tf_2_caffe/caffe_facenet_tensors.txt', 'w') as fp:
    fp.writelines(caffe_tensors)

In [4]:
caffe_net = caffe.Net('/home/ovuser/Projects/tf_2_caffe/facenet_deploy.prototxt', caffe.TEST)

In [ ]:
def get_frozen_graph(model_dir = '/home/ovuser/FaceRecognition/models/20170512-110547'):
    with tf.Graph().as_default():
        with tf.Session() as sess:
            meta_file, ckpt_file = facenet.get_model_filenames(os.path.expanduser(model_dir))
            facenet.load_model(model_dir)
            all_tensors = tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
            lines = []
            with open('/home/ovuser/Desktop/CaffeFaceResearch/Facenet_Tensors', 'w') as fp:
                for tensor in all_tensors:
                    try:
                        lines.append(tensor.name + ' '+ ','.join([str(it) for it in tensor.get_shape()]) + '\n')
                    except ValueError:
                        print (tensor.name)
                fp.writelines(lines)
            graphdef = tf.get_default_graph().as_graph_def()
            op = sess.graph.get_operations()
            writer = tf.summary.FileWriter("/home/ovuser/research/logs/facenet")
            frozen_graph = tf.graph_util.convert_variables_to_constants(sess, graphdef, ["embeddings"])
            writer.add_graph(sess.graph)
            return tf.graph_util.remove_training_nodes(frozen_graph), sess.graph

In [ ]:
print(caffe_net.params['conv1_3x3_s2'][0].data.shape)

In [2]:
conv_layers = {'InceptionResnetV1/Conv2d_1a_3x3/weights:0':'conv1_3x3_s2', 'InceptionResnetV1/Conv2d_2a_3x3/weights:0':'conv2_3x3_s1', 'InceptionResnetV1/Conv2d_2b_3x3/weights:0':'conv3_3x3_s1', 'InceptionResnetV1/Conv2d_3b_1x1/weights:0':'conv4_1x1_s1', 'InceptionResnetV1/Conv2d_4a_3x3/weights:0':'conv5_3x3_s1', 'InceptionResnetV1/Conv2d_4b_3x3/weights:0':'conv6_3x3_s2', 'InceptionResnetV1/Repeat/block35_1/Branch_0/Conv2d_1x1/weights:0':'inception_a1_1x1_b0', 'InceptionResnetV1/Repeat/block35_1/Branch_1/Conv2d_0a_1x1/weights:0':'inception_a1_1x1_b1', 'InceptionResnetV1/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/weights:0':'inception_a1_3x3_b1', 'InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0a_1x1/weights:0':'inception_a1_1x1_b2', 'InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/weights:0':'inception_a1_3x3_b2', 'InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0c_3x3/weights:0':'inception_a1_3x3_b2_a', 'InceptionResnetV1/Repeat/block35_1/Conv2d_1x1/weights:0':'inception_a1_1x1', 'InceptionResnetV1/Repeat/block35_2/Branch_0/Conv2d_1x1/weights:0':'inception_a2_1x1_b0', 'InceptionResnetV1/Repeat/block35_2/Branch_1/Conv2d_0a_1x1/weights:0':'inception_a2_1x1_b1', 'InceptionResnetV1/Repeat/block35_2/Branch_1/Conv2d_0b_3x3/weights:0':'inception_a2_3x3_b1', 'InceptionResnetV1/Repeat/block35_2/Branch_2/Conv2d_0a_1x1/weights:0':'inception_a2_1x1_b2', 'InceptionResnetV1/Repeat/block35_2/Branch_2/Conv2d_0b_3x3/weights:0':'inception_a2_3x3_b2', 'InceptionResnetV1/Repeat/block35_2/Branch_2/Conv2d_0c_3x3/weights:0':'inception_a2_3x3_b2_a', 'InceptionResnetV1/Repeat/block35_2/Conv2d_1x1/weights:0':'inception_a2_1x1', 'InceptionResnetV1/Repeat/block35_3/Branch_0/Conv2d_1x1/weights:0':'inception_a3_1x1_b0', 'InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/weights:0':'inception_a3_1x1_b1', 'InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0b_3x3/weights:0':'inception_a3_3x3_b1', 'InceptionResnetV1/Repeat/block35_3/Branch_2/Conv2d_0a_1x1/weights:0':'inception_a3_1x1_b2', 'InceptionResnetV1/Repeat/block35_3/Branch_2/Conv2d_0b_3x3/weights:0':'inception_a3_3x3_b2', 'InceptionResnetV1/Repeat/block35_3/Branch_2/Conv2d_0c_3x3/weights:0':'inception_a3_3x3_b2_a', 'InceptionResnetV1/Repeat/block35_3/Conv2d_1x1/weights:0':'inception_a3_1x1', 'InceptionResnetV1/Repeat/block35_4/Branch_0/Conv2d_1x1/weights:0':'inception_a4_1x1_b0', 'InceptionResnetV1/Repeat/block35_4/Branch_1/Conv2d_0a_1x1/weights:0':'inception_a4_1x1_b1', 'InceptionResnetV1/Repeat/block35_4/Branch_1/Conv2d_0b_3x3/weights:0':'inception_a4_3x3_b1', 'InceptionResnetV1/Repeat/block35_4/Branch_2/Conv2d_0a_1x1/weights:0':'inception_a4_1x1_b2', 'InceptionResnetV1/Repeat/block35_4/Branch_2/Conv2d_0b_3x3/weights:0':'inception_a4_3x3_b2', 'InceptionResnetV1/Repeat/block35_4/Branch_2/Conv2d_0c_3x3/weights:0':'inception_a4_3x3_b2_a', 'InceptionResnetV1/Repeat/block35_4/Conv2d_1x1/weights:0':'inception_a4_1x1', 'InceptionResnetV1/Repeat/block35_5/Branch_0/Conv2d_1x1/weights:0':'inception_a5_1x1_b0', 'InceptionResnetV1/Repeat/block35_5/Branch_1/Conv2d_0a_1x1/weights:0':'inception_a5_1x1_b1', 'InceptionResnetV1/Repeat/block35_5/Branch_1/Conv2d_0b_3x3/weights:0':'inception_a5_3x3_b1', 'InceptionResnetV1/Repeat/block35_5/Branch_2/Conv2d_0a_1x1/weights:0':'inception_a5_1x1_b2', 'InceptionResnetV1/Repeat/block35_5/Branch_2/Conv2d_0b_3x3/weights:0':'inception_a5_3x3_b2', 'InceptionResnetV1/Repeat/block35_5/Branch_2/Conv2d_0c_3x3/weights:0':'inception_a5_3x3_b2_a', 'InceptionResnetV1/Repeat/block35_5/Conv2d_1x1/weights:0':'inception_a5_1x1', 'InceptionResnetV1/Mixed_6a/Branch_0/Conv2d_1a_3x3/weights:0':'reduction_a_conv_3x3', 'InceptionResnetV1/Mixed_6a/Branch_1/Conv2d_0a_1x1/weights:0':'reduction_a_conv_1x1_reduce', 'InceptionResnetV1/Mixed_6a/Branch_1/Conv2d_0b_3x3/weights:0':'reduction_a_conv_3x3_2', 'InceptionResnetV1/Mixed_6a/Branch_1/Conv2d_1a_3x3/weights:0':'reduction_a_conv_3x3_3'}

In [18]:
print (len(conv_layers))

132


In [ ]:
with open('caffe_facenet_tensors.txt', 'r') as fp:
    caffe_facenet_tensors = fp.readlines()

In [ ]:
caffe_facenet_conv_layers = [tensor.replace('\n','') for tensor in caffe_facenet_tensors if 'conv' in tensor]

In [ ]:
print (caffe_facenet_conv_layers)

In [6]:
import json

In [17]:
with open('tf_2_caffe_conv_layers.json', 'w') as fp:
    json.dump(conv_layers, fp)

In [1]:
inc_block_b_conv_layers = {'InceptionResnetV1/Repeat_1/block17_#/Branch_0/Conv2d_1x1/weights:0':'inception_b#_1x1', 'InceptionResnetV1/Repeat_1/block17_#/Branch_1/Conv2d_0a_1x1/weights:0':'inception_b#_1x7_2_reduce', 'InceptionResnetV1/Repeat_1/block17_#/Branch_1/Conv2d_0b_1x7/weights:0':'inception_b#_1x7_2', 'InceptionResnetV1/Repeat_1/block17_#/Branch_1/Conv2d_0c_7x1/weights:0':'inception_b#_7x1_2', 'InceptionResnetV1/Repeat_1/block17_#/Conv2d_1x1/weights:0':'inception_b#_1x1_a'}

In [12]:
for i in range(1,6, 1):
    for k,v in inc_block_c_conv_layers.items():
        conv_layers[k.replace('#', str(i))] = v.replace('#', str(i))

In [8]:
red_block_b_conv_layers = {'InceptionResnetV1/Mixed_7a/Branch_0/Conv2d_0a_1x1/weights:0':'reduction_b_3x3_reduce', 'InceptionResnetV1/Mixed_7a/Branch_0/Conv2d_1a_3x3/weights:0':'reduction_b_3x3', 'InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/weights:0':'reduction_b_3x3_reduce_1', 'InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_1a_3x3/weights:0':'reduction_b_3x3_1', 'InceptionResnetV1/Mixed_7a/Branch_2/Conv2d_0a_1x1/weights:0':'reduction_b_3x3_reduce_2', 'InceptionResnetV1/Mixed_7a/Branch_2/Conv2d_0b_3x3/weights:0':'reduction_b_3x3_2', 'InceptionResnetV1/Mixed_7a/Branch_2/Conv2d_1a_3x3/weights:0':'reduction_b_3x3_2a'}
for k, v in red_block_b_conv_layers.items():
    conv_layers[k] = v

In [11]:
inc_block_c_conv_layers = {'InceptionResnetV1/Repeat_2/block8_#/Branch_0/Conv2d_1x1/weights:0':'inception_c#_1x1', 'InceptionResnetV1/Repeat_2/block8_#/Branch_1/Conv2d_0a_1x1/weights:0':'inception_c#_1x1_2', 'InceptionResnetV1/Repeat_2/block8_#/Branch_1/Conv2d_0b_1x3/weights:0':'inception_c#_1x3_2', 'InceptionResnetV1/Repeat_2/block8_#/Branch_1/Conv2d_0c_3x1/weights:0':'inception_c#_3x1_2', 'InceptionResnetV1/Repeat_2/block8_#/Conv2d_1x1/weights:0':'inception_c#_1x1_3'}

In [15]:
inc_last_block_conv_layers = {'InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/weights:0':'inception_c6_1x1', 'InceptionResnetV1/Block8/Branch_1/Conv2d_0a_1x1/weights:0':'inception_c6_1x1_2', 'InceptionResnetV1/Block8/Branch_1/Conv2d_0b_1x3/weights:0':'inception_c6_1x3_2', 'InceptionResnetV1/Block8/Branch_1/Conv2d_0c_3x1/weights:0':'inception_c6_3x1_2', 'InceptionResnetV1/Block8/Conv2d_1x1/weights:0':'inception_c6_1x1_3'}

In [16]:
for k, v in inc_last_block_conv_layers.items():
    conv_layers[k] = v

In [1]:
import re

In [2]:
re.match('asdf\d', 'asdf9')

In [4]:
re.match('asdf\d', 'asdf9') is None

False